In [1]:
import os
os.environ['OPENTSDB_PYTHON_METRICS_TEST_MODE'] = 'True'

In [2]:
from banzai.calibrations import make_master_calibrations
import requests
from banzai_floyds import settings
import banzai.dbs
from banzai.utils.stage_utils import run_pipeline_stages
import logging
from banzai.logs import set_log_level
from glob import glob

import pkg_resources
import banzai_floyds.dbs
from astropy.io import ascii, fits
from banzai.utils.fits_utils import download_from_s3

2023-09-07 16:29:54.713  WARNING:        warnings: /Users/cmccully/miniconda3/envs/banzai-floyds/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "

2023-09-07 16:29:55.372     INFO:           utils: Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-09-07 16:29:55.373     INFO:           utils: NumExpr defaulting to 8 threads.


In [3]:
os.environ['DB_ADDRESS'] = 'sqlite:///test_data/test.db'

settings.processed_path= os.path.join(os.getcwd(), 'test_data')
settings.fpack=True
settings.db_address = os.environ['DB_ADDRESS']
settings.reduction_level = 91
settings.ARCHIVE_API_ROOT = 'https://archive-api.lco.global/'
settings.RAW_DATA_API_ROOT = 'https://archive-api.lco.global/'
settings.RAW_DATA_FRAME_URL = 'https://archive-api.lco.global/frames'
settings.ARCHIVE_FRAME_URL = 'https://archive-api.lco.global/frames'

In [4]:
# set up the context object.
import banzai.main
context = banzai.main.parse_args(settings, parse_system_args=False)

In [11]:
os.makedirs('test_data', exist_ok=True)

In [23]:
banzai.dbs.create_db(os.environ["DB_ADDRESS"])
os.system(f'banzai_add_site --site ogg --latitude 20.7069444444 --longitude -156.258055556 --elevation 3065 --timezone -10 --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_site --site coj --latitude -31.272932 --longitude 149.070648 --elevation 1116 --timezone 10 --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_instrument --site ogg --camera en06 --name floyds01 --instrument-type 2m0-FLOYDS-SciCam --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_instrument --site coj --camera en12 --name floyds02 --instrument-type 2m0-FLOYDS-SciCam --db-address={os.environ["DB_ADDRESS"]}')
banzai_floyds.dbs.ingest_standards(os.environ["DB_ADDRESS"])

2023-08-08 14:43:23.983  WARNING:        warnings: /Users/cmccully/miniconda3/envs/banzai-floyds/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "

2023-08-08 14:43:24.617     INFO:           utils: Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-08-08 14:43:24.617     INFO:           utils: NumExpr defaulting to 8 threads.
2023-08-08 14:43:25.455  WARNING:        warnings: /Users/cmccully/miniconda3/envs/banzai-floyds/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "

2023-08-08 14:

In [37]:
skyflat_files = ascii.read(pkg_resources.resource_filename('banzai_floyds.tests', 'data/test_skyflat.dat'))
for skyflat in skyflat_files:
    skyflat_info = dict(skyflat)
    context = banzai.main.parse_args(settings, parse_system_args=False)
    skyflat_hdu = fits.open(download_from_s3(skyflat_info, context))

    # Munge the data to be OBSTYPE SKYFLAT
    skyflat_hdu['SCI'].header['OBSTYPE'] = 'SKYFLAT'
    skyflat_name = skyflat_info["filename"].replace("x00.fits", "f00.fits")
    filename = os.path.join('test_data', f'{skyflat_name}')
    skyflat_hdu.writeto(filename, overwrite=True)
    skyflat_hdu.close()
    run_pipeline_stages([{'path': filename}], context)

2023-08-08 14:50:21.963     INFO:      fits_utils: Downloading file ogg2m001-en06-20190329-0018-x00.fits.fz from archive. ID: 29878203. | {"filename": "ogg2m001-en06-20190329-0018-x00.fits.fz", "attempt_number": 1, "processName": "MainProcess"}
2023-08-08 14:50:24.537     INFO:          stages: Running banzai.stages.OverscanSubtractor | {"filename": "ogg2m001-en06-20190329-0018-f00.fits.fz", "site": "ogg", "instrument": "floyds01", "epoch": "20190329", "request_num": "UNSPECIFIED", "obstype": "SKYFLAT", "filter": "air", "processName": "MainProcess"}
2023-08-08 14:50:24.540     INFO:          stages: Running banzai.stages.Trimmer | {"filename": "ogg2m001-en06-20190329-0018-f00.fits.fz", "site": "ogg", "instrument": "floyds01", "epoch": "20190329", "request_num": "UNSPECIFIED", "obstype": "SKYFLAT", "filter": "air", "processName": "MainProcess"}
2023-08-08 14:50:24.541     INFO:            trim: Trimming image | {"filename": "ogg2m001-en06-20190329-0018-f00.fits.fz", "site": "ogg", "inst

In [5]:
DATA_FILELIST = pkg_resources.resource_filename('banzai_floyds.tests', 'data/test_data.dat')
test_data = ascii.read(DATA_FILELIST)

In [41]:
for row in test_data:
    if 'a00.fits' in row['filename']:
        archive_record = requests.get(f'{context.ARCHIVE_FRAME_URL}/{row["frameid"]}').json()
        archive_record['frameid'] = archive_record['id']
        run_pipeline_stages([archive_record], context)

2023-08-08 15:16:34.678     INFO:      fits_utils: Downloading file coj2m002-en12-20200813-0015-a00.fits.fz from archive. ID: 33851480. | {"filename": "coj2m002-en12-20200813-0015-a00.fits.fz", "attempt_number": 1, "processName": "MainProcess"}
2023-08-08 15:16:36.681     INFO:          stages: Running banzai.stages.OverscanSubtractor | {"filename": "coj2m002-en12-20200813-0015-a00.fits.fz", "site": "coj", "instrument": "floyds02", "epoch": "20200813", "request_num": "2207712", "obstype": "ARC", "filter": "air", "processName": "MainProcess"}
2023-08-08 15:16:36.684     INFO:          stages: Running banzai.stages.Trimmer | {"filename": "coj2m002-en12-20200813-0015-a00.fits.fz", "site": "coj", "instrument": "floyds02", "epoch": "20200813", "request_num": "2207712", "obstype": "ARC", "filter": "air", "processName": "MainProcess"}
2023-08-08 15:16:36.684     INFO:            trim: Trimming image | {"filename": "coj2m002-en12-20200813-0015-a00.fits.fz", "site": "coj", "instrument": "floyds

In [47]:
for row in test_data:
    if 'w00.fits' in row['filename']:
        archive_record = requests.get(f'{context.ARCHIVE_FRAME_URL}/{row["frameid"]}').json()
        archive_record['frameid'] = archive_record['id']
        run_pipeline_stages([archive_record], context)

2023-08-08 15:33:56.376     INFO:      fits_utils: Downloading file coj2m002-en12-20200813-0016-w00.fits.fz from archive. ID: 33851505. | {"filename": "coj2m002-en12-20200813-0016-w00.fits.fz", "attempt_number": 1, "processName": "MainProcess"}
2023-08-08 15:33:58.482     INFO:          stages: Running banzai.stages.OverscanSubtractor | {"filename": "coj2m002-en12-20200813-0016-w00.fits.fz", "site": "coj", "instrument": "floyds02", "epoch": "20200813", "request_num": "2207712", "obstype": "LAMPFLAT", "filter": "air", "processName": "MainProcess"}
2023-08-08 15:33:58.484     INFO:          stages: Running banzai.stages.Trimmer | {"filename": "coj2m002-en12-20200813-0016-w00.fits.fz", "site": "coj", "instrument": "floyds02", "epoch": "20200813", "request_num": "2207712", "obstype": "LAMPFLAT", "filter": "air", "processName": "MainProcess"}
2023-08-08 15:33:58.485     INFO:            trim: Trimming image | {"filename": "coj2m002-en12-20200813-0016-w00.fits.fz", "site": "coj", "instrument

In [7]:
coj_floyds = banzai.dbs.get_instruments_at_site('coj', settings.db_address)[0]
ogg_floyds = banzai.dbs.get_instruments_at_site('ogg', settings.db_address)[0]
make_master_calibrations(ogg_floyds, 'LAMPFLAT', '2020-01-01', '2021-01-01', context)
make_master_calibrations(coj_floyds, 'LAMPFLAT', '2020-01-01', '2021-01-01', context)

2023-09-05 11:31:12.412     INFO:    calibrations: Making master frames | {"type": "2m0-FLOYDS-SciCam", "site": "ogg", "camera": "en06", "obstype": "LAMPFLAT", "min_date": "2020-01-01", "max_date": "2021-01-01", "processName": "MainProcess"}
2023-09-05 11:31:12.578     INFO:          stages: Running banzai.stages.FringeMaker | {"filename": "ogg2m001-en06-20200822-0008-w91.fits.fz", "site": "ogg", "instrument": "floyds01", "epoch": "20200822", "request_num": "2217414", "obstype": "LAMPFLAT", "filter": "air", "processName": "MainProcess"}
2023-09-05 11:31:17.945  WARNING:        warnings: /Users/cmccully/miniconda3/envs/banzai-floyds/lib/python3.10/site-packages/astropy/io/fits/card.py:1008: VerifyWarning: Card is too long, comment will be truncated.
  warnings.warn('Card is too long, comment will be truncated.',

2023-09-05 11:31:18.125     INFO:    calibrations: Finished | {"processName": "MainProcess"}
2023-09-05 11:31:18.126     INFO:    calibrations: Making master frames | {"type": 

In [6]:
for row in test_data:
    if 'e00.fits' in row['filename']:
        archive_record = requests.get(f'{context.ARCHIVE_FRAME_URL}/{row["frameid"]}').json()
        archive_record['frameid'] = archive_record['id']
        run_pipeline_stages([archive_record], context)

2023-09-07 16:30:20.894     INFO:      fits_utils: Downloading file coj2m002-en12-20200813-0014-e00.fits.fz from archive. ID: 33851447. | {"filename": "coj2m002-en12-20200813-0014-e00.fits.fz", "attempt_number": 1, "processName": "MainProcess"}
2023-09-07 16:30:22.475     INFO:          stages: Running banzai.stages.OverscanSubtractor | {"filename": "coj2m002-en12-20200813-0014-e00.fits.fz", "site": "coj", "instrument": "floyds02", "epoch": "20200813", "request_num": "2207712", "obstype": "SPECTRUM", "filter": "air", "processName": "MainProcess"}
2023-09-07 16:30:22.478     INFO:          stages: Running banzai.stages.Trimmer | {"filename": "coj2m002-en12-20200813-0014-e00.fits.fz", "site": "coj", "instrument": "floyds02", "epoch": "20200813", "request_num": "2207712", "obstype": "SPECTRUM", "filter": "air", "processName": "MainProcess"}
2023-09-07 16:30:22.478     INFO:            trim: Trimming image | {"filename": "coj2m002-en12-20200813-0014-e00.fits.fz", "site": "coj", "instrument

ConnectionError: HTTPSConnectionPool(host='archive-api.lco.global', port=443): Max retries exceeded with url: /frames/33852323 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x163eab7c0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [7]:
%debug

> /Users/cmccully/miniconda3/envs/banzai-floyds/lib/python3.10/site-packages/requests/adapters.py(516)send()
    514                 raise SSLError(e, request=request)
    515 
--> 516             raise ConnectionError(e, request=request)
    517 
    518         except ClosedPoolError as e:



ipdb>  u/


*** Invalid frame count (/)


ipdb>  q


In [11]:
from astropy.io import fits

In [13]:
h = fits.Header({'foo': 'bar'})
h.pop('foo')
h